In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()



Saving BTC-USD.csv to BTC-USD (3).csv


In [2]:
import io 

df = pd.read_csv(io.BytesIO(uploaded['BTC-USD.csv']))

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-10-11,8585.262695,8721.780273,8316.181641,8321.756836,8321.756836,1.960438e+10
1,2019-10-12,8315.665039,8415.242188,8313.340820,8336.555664,8336.555664,1.453264e+10
2,2019-10-13,8336.902344,8470.988281,8276.612305,8321.005859,8321.005859,1.380829e+10
3,2019-10-14,8320.832031,8390.208984,8284.130859,8374.686523,8374.686523,1.515139e+10
4,2019-10-15,8373.458008,8410.714844,8182.706543,8205.369141,8205.369141,1.522041e+10


In [4]:
#Library Import

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [5]:
# Data PreProcessing

df = df.set_index("Date")[['Close']].tail(1000)
df = df.set_index(pd.to_datetime(df.index))

In [6]:
df.head()

,Close
Date,
2019-10-11,8321.756836
2019-10-12,8336.555664
2019-10-13,8321.005859
2019-10-14,8374.686523
2019-10-15,8205.369141


In [7]:
#Normalizing Data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

In [8]:
df.head()

,Close
Date,
2019-10-11,0.460069
2019-10-12,0.462101
2019-10-13,0.459966
2019-10-14,0.467336
2019-10-15,0.444090


In [9]:
#Helper Functions that will help us to learn and analyse our model.

In [17]:
def visualize_training_results(results):
    """
    Plots the loss and accuracy for the training and testing data
    """
    history = results.history
    plt.figure(figsize=(12,4))
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure(figsize=(12,4))
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [11]:
def split_sequence(seq, n_steps_in, n_steps_out):
    """
    Splits the univariate time sequence
    """
    X, y = [], []
    
    for i in range(len(seq)):
        end = i + n_steps_in
        out_end = end + n_steps_out
        
        if out_end > len(seq):
            break
        
        seq_x, seq_y = seq[i:end], seq[end:out_end]
        
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [12]:
def layer_maker(n_layers, n_nodes, activation, drop=None, d_rate=.5):
    """
    Create a specified number of hidden layers for an RNN
    Optional: Adds regularization option, dropout layer to prevent potential overfitting if necessary
    """
    
    # Creating the specified number of hidden layers with the specified number of nodes
    for x in range(1,n_layers+1):
        model.add(LSTM(n_nodes, activation=activation, return_sequences=True))

        # Adds a Dropout layer after every Nth hidden layer (the 'drop' variable)
        try:
            if x % drop == 0:
                model.add(Dropout(d_rate))
        except:
            pass

In [13]:
#Now we will split our data.

n_per_in = 30  # the no. of days we will take to train our model.

n_per_out =10 # the no. of days (future) for which we are predicing price.

# Features (in this case it's 1 because there is only one feature: price)
n_features = 1

X, y = split_sequence(list(df.Close), n_per_in, n_per_out)

# Reshaping the X variable from 2D to 3D
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [14]:
from keras.layers.core import Dense,Activation,Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.models import Sequential
model = Sequential()

activ = "softsign"

#Input Layer
model.add(LSTM(30, activation=activ, return_sequences=True, input_shape=(n_per_in, n_features)))

#Hidden Layer
layer_maker(n_layers=6, n_nodes=12, activation=activ)

#Last layer(activation)
model.add(LSTM(10, activation=activ))

model.add(Dense(n_per_out))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 30)            3840      
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 12)            2064      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 12)            1200      
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 12)            1

In [15]:
# Training of our Neural Network
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)
res = model.fit(X, y, epochs=300, batch_size=32, validation_split=0.1)

Epoch 1/300
10/10 [==============================] - 2s 185ms/step - loss: nan - val_loss: nan
Epoch 2/300
10/10 [==============================] - 1s 85ms/step - loss: nan - val_loss: nan
Epoch 3/300
10/10 [==============================] - 1s 83ms/step - loss: nan - val_loss: nan
Epoch 4/300
10/10 [==============================] - 1s 84ms/step - loss: nan - val_loss: nan
Epoch 5/300
10/10 [==============================] - 1s 83ms/step - loss: nan - val_loss: nan
Epoch 6/300
10/10 [==============================] - 1s 84ms/step - loss: nan - val_loss: nan
Epoch 7/300
10/10 [==============================] - 1s 88ms/step - loss: nan - val_loss: nan
Epoch 8/300
10/10 [==============================] - 1s 85ms/step - loss: nan - val_loss: nan
Epoch 9/300
10/10 [==============================] - 1s 85ms/step - loss: nan - val_loss: nan
Epoch 10/300
10/10 [==============================] - 1s 83ms/step - loss: nan - val_loss: nan
Epoch 11/300
10/10 [==============================] - 1s 8